<a href="https://colab.research.google.com/github/IlyaDenisov88/dataenj/blob/main/PySpark/SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=bf6fe787cffcc12527201a3203f82692289984c9098620d0e231cb447236482c
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Пример использования DataFrame API


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Создание SparkSession
spark = SparkSession.builder.appName("DataFrameAPIExample").getOrCreate()

# Чтение данных из JSON файла
df = spark.read.json("/content/drive/MyDrive/dataenj/people.json")

# Фильтрация данных
filtered_df = df.filter(col("age") > 30) # тут он вывел

# Группировка и агрегация данных
grouped_df = df.groupBy("department").agg({"age": "avg", "name": "count"}).withColumnRenamed("avg(age)", "avg_age").withColumnRenamed("count(name)", "count")

# Сортировка данных
sorted_df = grouped_df.orderBy(col("count").desc()) # и тут он вывел

# Показ результатов
filtered_df.show()
sorted_df.show()

# Сохранение результирующего DataFrame в CSV файл
sorted_df.write.csv("output.csv", header=True, mode="overwrite")

spark.stop()

+---------------+---+----------+----+
|_corrupt_record|age|department|name|
+---------------+---+----------+----+
|           NULL| 35|        HR|Jane|
|           NULL| 40|   Finance|Mark|
+---------------+---+----------+----+

+-----------+-----+-------+
| department|count|avg_age|
+-----------+-----+-------+
|         HR|    2|   32.5|
|    Finance|    2|   32.5|
|Engineering|    1|   23.0|
|       NULL|    0|   NULL|
+-----------+-----+-------+



Теперь давайте посмотрим на то, как работает "оригинальный SQL" в PySpark.


In [ ]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("SQLAPIExample").getOrCreate()

# Чтение данных из JSON файлов
people_df = spark.read.json("/content/drive/MyDrive/dataenj/people.json")
departments_df = spark.read.json("/content/drive/MyDrive/dataenj/departments.json")

# Регистрация DataFrame как временные таблицы
people_df.createOrReplaceTempView("people")
departments_df.createOrReplaceTempView("departments")

# Выполнение JOIN-запроса с использованием SQL
join_df = spark.sql("""
SELECT p.name, p.age, d.department_name, d.id
FROM people p
JOIN departments d
ON p.department = d.department_name
""")

# Показ результатов
join_df.show()

# Сохранение результирующего DataFrame в CSV файл
join_df.write.csv("output2.csv", header=True, mode="overwrite")

spark.stop()

+-----+---+---------------+---+
| name|age|department_name| id|
+-----+---+---------------+---+
| John| 30|             HR|  1|
|  Doe| 25|        Finance|  2|
| Jane| 35|             HR|  1|
| Mark| 40|        Finance|  2|
|Smith| 23|    Engineering|  3|
+-----+---+---------------+---+



Примеры разных Join`ов


В PySpark доступно несколько соединений. Давайте разбирать их с примерами.

Для начала подготовим примеры данных.

In [ ]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("JoinExamples").getOrCreate()

# Пример данных для DataFrame people
people_data = [
    ("John", 30, 1),
    ("Doe", 25, 2),
    ("Jane", 35, 1),
    ("Mark", 40, 2),
    ("Smith", 23, 3)
]
people_columns = ["name", "age", "department_id"]
people_df = spark.createDataFrame(data=people_data, schema=people_columns)

# Пример данных для DataFrame departments
departments_data = [
    (1, "HR"),
    (2, "Finance"),
    (3, "Engineering"),
    (4, "Marketing")
]
departments_columns = ["id", "department_name"]
departments_df = spark.createDataFrame(data=departments_data, schema=departments_columns)

# Показ данных
people_df.show()
departments_df.show()

+-----+---+-------------+
| name|age|department_id|
+-----+---+-------------+
| John| 30|            1|
|  Doe| 25|            2|
| Jane| 35|            1|
| Mark| 40|            2|
|Smith| 23|            3|
+-----+---+-------------+

+---+---------------+
| id|department_name|
+---+---------------+
|  1|             HR|
|  2|        Finance|
|  3|    Engineering|
|  4|      Marketing|
+---+---------------+



Внутреннее соединение (Inner Join)


In [ ]:
# Внутреннее соединение
inner_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "inner")
inner_join_df.show()

+-----+---+-------------+---+---------------+
| name|age|department_id| id|department_name|
+-----+---+-------------+---+---------------+
| John| 30|            1|  1|             HR|
| Jane| 35|            1|  1|             HR|
|  Doe| 25|            2|  2|        Finance|
| Mark| 40|            2|  2|        Finance|
|Smith| 23|            3|  3|    Engineering|
+-----+---+-------------+---+---------------+



Левое внешнее соединение (Left Outer Join)


In [ ]:
# Левое внешнее соединение
left_outer_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "left_outer")
left_outer_join_df.show()

+-----+---+-------------+---+---------------+
| name|age|department_id| id|department_name|
+-----+---+-------------+---+---------------+
| John| 30|            1|  1|             HR|
|  Doe| 25|            2|  2|        Finance|
| Jane| 35|            1|  1|             HR|
|Smith| 23|            3|  3|    Engineering|
| Mark| 40|            2|  2|        Finance|
+-----+---+-------------+---+---------------+



Правое внешнее соединение (Right Outer Join)


In [ ]:
# Правое внешнее соединение
right_outer_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "right_outer")
right_outer_join_df.show()

+-----+----+-------------+---+---------------+
| name| age|department_id| id|department_name|
+-----+----+-------------+---+---------------+
| Jane|  35|            1|  1|             HR|
| John|  30|            1|  1|             HR|
| Mark|  40|            2|  2|        Finance|
|  Doe|  25|            2|  2|        Finance|
|Smith|  23|            3|  3|    Engineering|
| NULL|NULL|         NULL|  4|      Marketing|
+-----+----+-------------+---+---------------+



Полное внешнее соединение (Full Outer Join)


In [ ]:
full_outer_join_df = people_df.join(departments_df, people_df.department_id == departments_df.id, "outer")
full_outer_join_df.show()

+-----+----+-------------+---+---------------+
| name| age|department_id| id|department_name|
+-----+----+-------------+---+---------------+
| John|  30|            1|  1|             HR|
| Jane|  35|            1|  1|             HR|
|  Doe|  25|            2|  2|        Finance|
| Mark|  40|            2|  2|        Finance|
|Smith|  23|            3|  3|    Engineering|
| NULL|NULL|         NULL|  4|      Marketing|
+-----+----+-------------+---+---------------+



Полное перекрестное соединение (Cross Join)


In [ ]:
cross_join_df = people_df.crossJoin(departments_df)
cross_join_df.show()

+-----+---+-------------+---+---------------+
| name|age|department_id| id|department_name|
+-----+---+-------------+---+---------------+
| John| 30|            1|  1|             HR|
| John| 30|            1|  2|        Finance|
|  Doe| 25|            2|  1|             HR|
|  Doe| 25|            2|  2|        Finance|
| John| 30|            1|  3|    Engineering|
| John| 30|            1|  4|      Marketing|
|  Doe| 25|            2|  3|    Engineering|
|  Doe| 25|            2|  4|      Marketing|
| Jane| 35|            1|  1|             HR|
| Jane| 35|            1|  2|        Finance|
| Mark| 40|            2|  1|             HR|
| Mark| 40|            2|  2|        Finance|
|Smith| 23|            3|  1|             HR|
|Smith| 23|            3|  2|        Finance|
| Jane| 35|            1|  3|    Engineering|
| Jane| 35|            1|  4|      Marketing|
| Mark| 40|            2|  3|    Engineering|
| Mark| 40|            2|  4|      Marketing|
|Smith| 23|            3|  3|    E

Соединение с использованием условия (Join with Condition)


In [ ]:
condition_join_df = people_df.join(departments_df, (people_df.department_id == departments_df.id) & (people_df.age > 30), "inner")
condition_join_df.show()

+----+---+-------------+---+---------------+
|name|age|department_id| id|department_name|
+----+---+-------------+---+---------------+
|Jane| 35|            1|  1|             HR|
|Mark| 40|            2|  2|        Finance|
+----+---+-------------+---+---------------+

